In [ ]:
# import os
# os.environ['https_proxy'] = "http://hpc-proxy00.city.ac.uk:3128"
# # os.environ['http_proxy'] = “http://hpc-proxy00.city.ac.uk:3128”

In [1]:
import numpy as np
from numpy.random import default_rng
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import PIL
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torch.utils.data.dataloader as dataloader
from torchvision import datasets
import torchvision.models as models
# from torchinfo import summary

In [2]:
transform_train = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
transform_test = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),     
            ])

In [3]:
filepath='.'
filedir=os.path.join(os.getcwd(),filepath,'Landscape Classification')
batch_size=16

# dataset=datasets.ImageFolder(
#     root=filedir,
#     transform=transform_train,
# )

# train_dataset,test_dataset,valid_dataset=torch.utils.data.random_split(dataset, [0.8,0.1,0.1],
# generator=torch.Generator().manual_seed(42))

for i in os.listdir(filedir):
    if(i=='Training Data'):
        train_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_train
    )
    elif(i=='Testing Data'):
        test_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_test
    )
    elif(i=='Validation Data'):
        valid_dataset=datasets.ImageFolder(
            root=os.path.join(filedir,i),
            transform=transform_test
    )
print(train_dataset)
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
valid_loader=DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

Dataset ImageFolder
    Number of datapoints: 10000
    Root location: c:\Users\Sean\Documents\GitHub\INM702-Coursework\.\Landscape Classification\Training Data
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


# Base Model

In [4]:
D1=32
D2=64
D3=128

A1=512
A2=256
num_classes=5
input_pix=224
num_neurons=int(np.floor(np.floor(np.floor(input_pix/2)/2)/2)**2*D3)
input=3
class my_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(input, D1, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(D1, D2, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(D2, D3, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            
            
            nn.Linear(num_neurons,A1),
            nn.ReLU(),
            nn.Linear(A1, A2),
            nn.ReLU(),
            nn.Linear(A2,num_classes)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [16]:
#Loading checkpoint in case of outside problems interrupting training
mydir='MODELS'
save_path=os.path.join(mydir, 'mymodel.pt')

device = ('cuda' if torch.cuda.is_available() else 'cpu')
lr = 1e-3
epochs = 15
opt=torch.optim.Adam
model=my_nn().to(device)

# if not os.path.isdir(mydir):
#     os.makedirs(mydir)

# if os.path.isfile(save_path):
#     checkpoint=torch.load(save_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     model.to(device)
#     opt.load_state_dict(checkpoint['optimizer_state_dict'])
#     start=checkpoint['epoch']
#     best_valid=checkpoint['valid_acc']
#     best_test=checkpoint['test_acc']
#     print('Starting from epoch', start)
# else:
#     print('No Checkpoint, starting from scratch')


In [6]:
def train(model,train_loader,epochs,lr,opt,loss_func):
    model.train()
    losslog=[]
    optimizer=opt
    for _,(x,y) in enumerate(train_loader):
        batch_x=x.to(device)
        ypred=model(batch_x)
        loss=loss_func(ypred,y.to(device))
        losslog.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return losslog

def evaluate(model,test_loader):
    model.eval()
    acc = 0
        
    with torch.no_grad():
        for _, (x, y) in enumerate(test_loader):
            batch_x = model(x.to(device))
            acc += (batch_x.argmax(1) == y.to(device)).sum().item()
            
    #return the accuracy from the epoch     
    return acc / len(test_loader.dataset)
            

def fit(model,train_loader,test_loader,val_loader,epochs,optimizer,lr=1e-3,loss_func=F.cross_entropy):
    train_acc_log=[]
    test_acc_log=[]
    valid_acc_log=[]
    best_valid=0
    best_test=0
    opt=optimizer(model.parameters(),lr)
    for iter in range(epochs):
        #training step
        train_log=train(model,train_loader,epochs,lr,opt,loss_func)
            
        train_acc=evaluate(model,train_loader)
        test_acc=evaluate(model,test_loader)
        valid_acc=evaluate(model,test_loader)
        print("Epoch [{}], train acc: {:.2f}, test acc: {:.2f}, val acc: {:.2f}".format(iter, train_acc, test_acc, valid_acc))
        #evaluate step
        train_acc_log.append(train_acc)
        test_acc_log.append(test_acc)
        valid_acc_log.append(valid_acc)
        
        # #saving models to avoid running through entire epochs
        # if(valid_acc>best_valid and test_acc>=best_test):
        #     best_valid=valid_acc
        #     best_test=test_acc
            
        #     torch.save(
        #         {
        #             'epochs': epochs,
        #             'model_state_dict': model.state_dict(),
        #             'opt_state_dict': opt.state_dict(),
        #             'train_acc': train_acc,
        #             'test_acc': test_acc,
        #             'valid_acc': valid_acc,
        #         }, save_path
        #     )
        #     print('Model Saved for epoch {}'.format(iter))
    
    
    print('Training Complete')
    return train_acc,test_acc,valid_acc



In [18]:
train_acc,test_acc,valid_acc=fit(model,train_loader,test_loader,valid_loader,epochs,opt)

Epoch [0], train acc: 0.71, test acc: 0.71, val acc: 0.71
Epoch [1], train acc: 0.74, test acc: 0.72, val acc: 0.72
Epoch [2], train acc: 0.74, test acc: 0.72, val acc: 0.72
Epoch [3], train acc: 0.87, test acc: 0.79, val acc: 0.79
Epoch [4], train acc: 0.93, test acc: 0.76, val acc: 0.76
Epoch [5], train acc: 0.97, test acc: 0.79, val acc: 0.79
Epoch [6], train acc: 0.98, test acc: 0.80, val acc: 0.80
Epoch [7], train acc: 0.99, test acc: 0.78, val acc: 0.78
Epoch [8], train acc: 0.98, test acc: 0.78, val acc: 0.78
Epoch [9], train acc: 0.99, test acc: 0.76, val acc: 0.76
Epoch [10], train acc: 0.99, test acc: 0.79, val acc: 0.79
Epoch [11], train acc: 0.99, test acc: 0.79, val acc: 0.79
Epoch [12], train acc: 0.98, test acc: 0.78, val acc: 0.78
Epoch [13], train acc: 0.99, test acc: 0.79, val acc: 0.79
Epoch [14], train acc: 0.99, test acc: 0.79, val acc: 0.79
Training Complete


# Transfer Learning

In [13]:
transfer_alexnet = models.alexnet(weights='DEFAULT').to(device)
for layers in transfer_alexnet.parameters():
    layers.requires_grad=False #Freeze layers

dropout=0.2
num_classes=5
transfer_alexnet.classifier= nn.Sequential( #Rebuilding the classifier to be the same as Alex Net but an added Dense layer for the number of classes for this dataset (5)
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(1000,num_classes)
        ).to(device)

# summary(transfer_alexnet, input_size=(batch_size,3,224,224),col_names=["input_size","output_size","trainable","num_params"])
        

Layer (type:depth-idx)                   Input Shape               Output Shape              Trainable                 Param #
AlexNet                                  [16, 3, 224, 224]         [16, 5]                   Partial                   --
├─Sequential: 1-1                        [16, 3, 224, 224]         [16, 256, 6, 6]           False                     --
│    └─Conv2d: 2-1                       [16, 3, 224, 224]         [16, 64, 55, 55]          False                     (23,296)
│    └─ReLU: 2-2                         [16, 64, 55, 55]          [16, 64, 55, 55]          --                        --
│    └─MaxPool2d: 2-3                    [16, 64, 55, 55]          [16, 64, 27, 27]          --                        --
│    └─Conv2d: 2-4                       [16, 64, 27, 27]          [16, 192, 27, 27]         False                     (307,392)
│    └─ReLU: 2-5                         [16, 192, 27, 27]         [16, 192, 27, 27]         --                        --
│    └

In [17]:
train_acc,test_acc,valid_acc=fit(transfer_alexnet,train_loader,test_loader,valid_loader,epochs,opt)

Epoch [0], train acc: 0.96, test acc: 0.85, val acc: 0.85
Epoch [1], train acc: 0.93, test acc: 0.86, val acc: 0.86
Epoch [2], train acc: 0.96, test acc: 0.87, val acc: 0.87
Epoch [3], train acc: 0.96, test acc: 0.85, val acc: 0.85
Epoch [4], train acc: 0.97, test acc: 0.86, val acc: 0.86
Epoch [5], train acc: 0.97, test acc: 0.85, val acc: 0.85
Epoch [6], train acc: 0.98, test acc: 0.85, val acc: 0.85
Epoch [7], train acc: 0.98, test acc: 0.82, val acc: 0.82
Epoch [8], train acc: 0.98, test acc: 0.86, val acc: 0.86
Epoch [9], train acc: 0.99, test acc: 0.84, val acc: 0.84
Epoch [10], train acc: 0.99, test acc: 0.86, val acc: 0.86
Epoch [11], train acc: 0.99, test acc: 0.85, val acc: 0.85
Epoch [12], train acc: 0.98, test acc: 0.88, val acc: 0.88
Epoch [13], train acc: 0.99, test acc: 0.85, val acc: 0.85
Epoch [14], train acc: 0.98, test acc: 0.87, val acc: 0.87
Training Complete
